In [1]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation

pd.options.display.float_format = '{:,.14f}'.format

In [ ]:
load_dotenv() #add external key request for user access

slug_collection = 'boredape-baseclub'
initial_token = 'LWV2ZW50X3RpbWVzdGFtcD0yMDI1LTAyLTExKzEyJTNBMTQlM0EzNi40NzQ4ODkmLWV2ZW50X3R5cGU9Y29sbGVjdGlvbl9vZmZlciYtcGs9MzE5Nzc0MTM2Nzk=' #first pagination 
event_type = 'order'
limit_request = 50

url = f'https://api.opensea.io/api/v2/events/collection/{slug_collection}?event_type={event_type}&limit={limit_request}'
api_key = os.getenv('api_key')

headers = {
  "accept": "application/json",
    "x-api-key": api_key  
}

<h5> Creating first request without pagination

In [16]:
first_response = requests.get(url, headers=headers)
data = first_response.json()

datasource_dir = "datasource"
orders_dir = os.path.join(datasource_dir,"orders")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"events_batch_0_{timestamp}.json"

if not os.path.exists(datasource_dir):
    os.makedirs(datasource_dir)
if not os.path.exists(orders_dir):
    os.makedirs(orders_dir)

filepath = os.path.join(orders_dir,filename)

with open(filepath,"w") as f:
    json.dump(data,f)


         


<h5> Creating all struture to collect all orders request with pagination

In [ ]:
def main_request(url, pagination_token, headers): 
    try:
        response = requests.get(url + f'&next={pagination_token}', headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error request: {e}")
        return None

In [ ]:
def batch_process(batch_size=2, delay=10,max_requests=50): #batch_process function is responsible for run requests in batch, according to interval
    base_dir = "datasource"
    orders_dir = os.path.join(base_dir, "orders")

    #Creating filepath
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        print(f"Pasta '{base_dir}' criada.")
    #Creating filepath
    if not os.path.exists(orders_dir):
        os.makedirs(orders_dir)
        print(f"Pasta '{base_dir}' criada.")

    current_token = initial_token
    batch_number = 1
    total_paginas_processadas = 0

    #Looping condition: While having 'asset_event' into request the function will create new requests, updating the pagination token
    #For each new request the result will be adding into "all_event" list 
    #When the last pagination is the same as "current_token" OR  don't having pagination the function break
    #All request will become JSON files and will be saved. Those file will be used into Power BI project
    while True:
        all_events = []
        request_count = 0

        for _ in range(batch_size):
            data = main_request(url,current_token, headers)
            if not data or 'asset_events' not in data:
                print("End of data. Error in request")
                return
            
            all_events.extend(data['asset_events'])
            next_token = data.get('next', None)

            if not next_token or next_token == current_token:
                print("There no pagination avaliable")
                if all_events:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"events_batch_{batch_number}_{timestamp}.json"
                    filepath = os.path.join(orders_dir,filename)
                    with open(filepath, 'w') as f:
                        json.dump(all_events, f)
                    print(f"✅ Final batch saved in {filename} (Events: {len(all_events)})")
                return  

            current_token = next_token
            request_count += 1
            total_paginas_processadas += 1

            # Break after X requests
            if request_count % max_requests == 0:
                print(f"⏸ Breack of {delay} segunds...")
                time.sleep(delay)

        # Saving the request as JSON file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"events_batch_{batch_number}_{timestamp}.json"
        filepath = os.path.join(orders_dir, filename)
        with open(filepath, 'w') as f:
            json.dump(all_events, f)
        print(f"✅ Batch {batch_number} saved in {filepath} (Events: {len(all_events)})")

        batch_number += 1
        time.sleep(delay)  

batch_process(batch_size=2, delay=10, max_requests=10)



In [3]:
pasta_json = 'datasource/orders'

dataframes = []

#making validation to collect only json files and adding into dataframes list 
for arquivo in os.listdir(pasta_json):
    if arquivo.endswith('.json'):
        caminho_completo = os.path.join(pasta_json, arquivo)

        df = pd.read_json(caminho_completo)
        dataframes.append(df)

df_orders_raw = pd.concat(dataframes, ignore_index= True)

#Creating dataframes through dictionaries series
df_asset = df_orders_raw['asset'].apply(pd.Series)
df_payment = df_orders_raw['payment'].apply(pd.Series)
df_criteria = df_orders_raw['criteria'].apply(pd.Series)
df_criteria_collection = df_criteria['collection'].apply(pd.Series)
df_criteria_contract = df_criteria['contract'].apply(pd.Series)

In [ ]:

#Adding datas from dataframes above into main dataframe
df_orders = pd.concat([
    df_orders_raw.drop(columns=['asset','payment','criteria']),
    df_asset.add_prefix('asset_'),
    df_payment.add_prefix('payment_'),
    df_criteria.add_prefix('criteria_'),
    df_criteria_collection.add_prefix('criteria-collection_'),
    df_criteria_contract.add_prefix('criteria-collection_')
],axis=1)


df_orders['start_date'] = pd.to_datetime(df_orders['start_date'],unit='s')
df_orders['start_date'] = df_orders['start_date'].dt.strftime('%d-%m-%Y')
df_orders['expiration_date'] = pd.to_datetime(df_orders['expiration_date'],unit='s')
df_orders['expiration_date'] = df_orders['expiration_date'].dt.strftime('%d-%m-%Y')


In [ ]:
df_orders['payment_quantity_characters'] = df_orders['payment_quantity'].astype(str).str.len()
df_orders['payment_qtd'] = df_orders['payment_quantity'].astype(str).str[:12]
df_orders['payment_qtd'] = pd.to_numeric(df_orders['payment_qtd'], errors='coerce')


def calcular_payment_value(row):
    qtd_value = row['payment_quantity_characters'] - row['payment_decimals']
    if row['payment_quantity_characters'] > row['payment_decimals']:
        return int(str(row['payment_qtd'])[:qtd_value])
    elif row['payment_quantity_characters'] < row['payment_decimals']:
        return float(row['payment_qtd']/10**12)

df_orders['payment_value'] = df_orders.apply(calcular_payment_value,axis=1)

In [11]:
datasource_dir = "datasource"
orders_dir = os.path.join(datasource_dir,"orders")
filename = f"{slug_collection}_{event_type}.csv"

if not os.path.exists(datasource_dir):
    os.makedirs(datasource_dir)
if not os.path.exists(orders_dir):
    os.makedirs(orders_dir)

filepath = os.path.join(orders_dir,filename)

df_orders.to_csv(filepath, index=False, encoding="utf-8")                

